In [1]:
%matplotlib inline

from __future__ import print_function
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import googlemaps  # pip install -U googlemaps
from geopy.distance import geodesic
import requests
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#sklearn libraries
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

In [2]:
user_evaluation = pd.read_csv('../survey/User_Evaluation_20180801.csv')

def user_input(userID):
    property_list = ['20174233022047', '20174151021003', '20174216015034', \
                     '20178706014038', '20174156016063', '20174339007052', \
                     '20172841038054', '20172728020017', '20177564020019', '20174230008070']
    property_list = list(map(int, property_list))
    
    input = user_evaluation.iloc[userID,17:30]
    ranks = user_evaluation.iloc[userID,30:-1]
    ranks = ranks.astype(int)
    
    property_ranks = pd.concat([pd.Series(property_list).reset_index(), \
                                ranks.reset_index()], axis=1, ignore_index=True)
    property_ranks = property_ranks.loc[:,[1,3]]
    property_ranks.columns = ['rowID', 'user_ranks']

    LA = user_evaluation.iloc[userID,-1]
    return [input, property_ranks, LA]

In [21]:
user_evaluation

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q25_2,Q25_3,Q25_4,Q25_5,Q25_8,Q25_10,Q25_11,Q25_12,Q25_14,Q26
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Please drag and drop to rank the following LA ...,Are you familiar with the LA area / housing ma...
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID42_2""}","{""ImportId"":""QID42_3""}","{""ImportId"":""QID42_4""}","{""ImportId"":""QID42_5""}","{""ImportId"":""QID42_6""}","{""ImportId"":""QID42_7""}","{""ImportId"":""QID42_8""}","{""ImportId"":""QID42_9""}","{""ImportId"":""QID42_10""}","{""ImportId"":""QID43""}"
2,2018-07-21 07:32:58,2018-07-21 07:38:53,IP Address,73.231.61.32,100,354,True,2018-07-21 07:38:53,R_3MyuNsvUcxldqrd,NaN,...,9,3,2,1,5,4,6,8,10,No
3,2018-07-21 08:01:53,2018-07-21 08:07:13,IP Address,47.144.182.208,100,319,True,2018-07-21 08:07:13,R_3iPR4GtZRwVtfTZ,NaN,...,1,2,9,5,3,10,7,8,4,Yes
4,2018-07-21 09:52:36,2018-07-21 10:01:33,IP Address,174.206.4.153,100,536,True,2018-07-21 10:01:33,R_2s6aNZh4LWKBX2n,NaN,...,9,4,7,3,8,1,5,10,2,No
5,2018-07-21 10:14:13,2018-07-21 10:20:54,IP Address,108.91.174.190,100,401,True,2018-07-21 10:20:55,R_3Gx502esDuCt7I7,NaN,...,7,3,5,4,2,6,9,1,8,No
6,2018-07-21 10:20:58,2018-07-21 10:28:00,IP Address,172.56.27.81,100,421,True,2018-07-21 10:28:00,R_A4dXnP4Wv5Y70Mp,NaN,...,8,1,6,7,2,10,4,9,3,Yes
7,2018-07-21 10:40:44,2018-07-21 10:49:39,IP Address,98.176.184.165,100,535,True,2018-07-21 10:49:40,R_3htHOhTkffdEDoS,NaN,...,3,5,2,6,8,4,1,9,10,No
8,2018-07-21 12:42:58,2018-07-21 12:46:56,IP Address,71.183.192.233,100,238,True,2018-07-21 12:46:57,R_9uFaJKLFhZyyNH3,NaN,...,3,9,4,2,10,5,6,8,1,No
9,2018-07-22 07:32:12,2018-07-22 07:53:03,IP Address,45.56.155.191,100,1250,True,2018-07-22 07:53:03,R_2D6Vmi2G9z1t45T,NaN,...,10,9,5,3,6,1,4,7,2,No


In [10]:
def read_data():
    # read original dataset and standardized dataset
    df_transform = pd.read_csv('Model_output/df_transform_July2016.csv')
    df = pd.read_csv('Simple Model with Integrated Dataframe/df_integrated_0714.csv')
    property_list = ['20174233022047', '20174151021003', '20174216015034', \
                  '20178706014038', '20174156016063', '20174339007052', \
                  '20172841038054', '20172728020017', '20177564020019', '20174230008070']
    property_list = list(map(int, property_list))
    df = df[df['rowID'].isin(property_list)] 
    propertyIndex = dict(zip(property_list,range(len(property_list))))
    df['survey_order'] = df['rowID'].map(propertyIndex) + 1       
    df_transform = df_transform.iloc[df.index, :]
    df_features = ['zip_rank', 'SQFTmain', 'Units', 'Bedrooms', 'EffectiveYearBuilt', \
                    'house', 'condo', 'pud', 'pool', 'HH_Kids', 'Owner', 'lot_area', \
                    'num_school_choices', 'closest_school_rating', 'geography', 'parks', 'groceries', 'walkscore', 'transit_score']
    personal_features = ['SQFTmain', 'lot_area', 'EffectiveYearBuilt', \
                         'closest_school_rating', 'HH_Kids', 'Owner', 'crime', 'geography', \
                         'walkscore', 'transit_score',  'closest_schools', 'groceries', 'parks']
    return [df, df_transform, df_features, personal_features]

In [82]:
def model_output_ranks(model, userID, price_wt):
    user_numbers = user_input(userID)[0]
    user_numbers = list(map(int, user_numbers))
    user_scores = list(np.array(user_numbers[0:3])/sum(user_numbers[0:3]) * 100) + \
                        list(np.array(user_numbers[3:8])/sum(user_numbers[3:8]) * 100) + \
                        list(np.array(user_numbers[8:13])/sum(user_numbers[8:13]) * 100)
        
    def rf_score(price_wt, df, personal_wt):
        rf_score = price_wt * df['TotalValue']
        df_feature_score = pd.read_csv('Model_output/df_feature_score.csv')
        for feature in personal_wt.keys():
            rf_score += df_feature_score[feature] * personal_wt[feature]
        return rf_score

    def personal_listings_rf(price_wt, personal_scores):
        personal_dict = {}
        personal_wt = {}

        df, df_transform, df_features, personal_features = read_data()

        for i in range(len(personal_scores)):
            personal_dict[personal_features[i]] = personal_scores[i]
        for f in df_features:
            if f in personal_dict.keys():
                personal_wt[f] = personal_dict[f] / 100
            else:
                personal_wt[f] = 0

        rf_scores = rf_score(price_wt, df_transform, personal_wt)

        for i in df.index:
            df.loc[i, 'rf_scores'] = rf_scores[i]

        return df.sort_values(by=['rf_scores'], ascending=False)
    
    def poly_personal(data, personal_wt, row):
        poly_coef = np.loadtxt('../../test/poly_coef.out', delimiter=',')
        total = 0
        features = data.columns
        feature_number = len(features)
        sq_indices = [feature_number + 1]
        for i in range(feature_number, 1, -1):
            sq_indices.append(sq_indices[-1] + i)
        for i in range(feature_number):
            total += poly_coef[i + 1] * data.loc[row,features[i]] * personal_wt[i]
            total += poly_coef[sq_indices[i]] * (data.loc[row,features[i]] ** 2) * (personal_wt[i] ** 2)
            for j in range(i+1, feature_number):
                total += poly_coef[sq_indices[i] + j - i] * data.loc[row,features[i]] * data.loc[row,features[j]] * personal_wt[i] * personal_wt[j]
        return total

    def personal_listings_poly(price_wt, personal_scores):
        personal_dict = {}
        personal_wt = []

        df, df_transform, df_features, personal_features = read_data()

        for i in range(len(personal_scores)):
            personal_dict[personal_features[i]] = personal_scores[i]     
        for f in df_features:
            if f in personal_dict.keys():
                personal_wt.append(personal_dict[f] / 100)
            else:
                personal_wt.append(0)

        X = df_transform[df_features]
        
        for i in df.index:
            df.loc[i, 'poly_scores'] = price_wt * df_transform.loc[i, 'TotalValue'] + 10**(-8) * poly_personal(X, np.array(personal_wt), i)
        return df.sort_values(by=['poly_scores'], ascending=False)


    def get_rf_top10(price_wt):
        personal_df = personal_listings_rf(price_wt, user_scores).reset_index()
        personal_df['model_ranks'] = personal_df['rf_scores'].rank(ascending=False).astype(int)
        return personal_df
    
    def get_poly_top10(price_wt):
        personal_df = personal_listings_poly(price_wt, user_scores).reset_index()
        personal_df['model_ranks'] = personal_df['poly_scores'].rank(ascending=False).astype(int)
        return personal_df
    
    def get_ensemble_top10(price_wt):
        df_poly = get_poly_top10(10000)
        df_rf = get_rf_top10(price_wt)
        personal_df = pd.merge(left = df_rf, right=df_poly, left_on = 'rowID', right_on = 'rowID')
        personal_df['combined_ranks'] = personal_df['model_ranks_x'] + personal_df['model_ranks_y']
        personal_df['model_ranks'] = personal_df['combined_ranks'].rank(ascending=True).astype(int)
        personal_df['survey_order'] = personal_df['survey_order_x']
        return personal_df
    
    if model == 'rf':
        return get_rf_top10(price_wt)[['rowID', 'survey_order', 'model_ranks']]
    if model == 'poly':
        return get_poly_top10(price_wt)[['rowID', 'survey_order', 'model_ranks']]
    if model == 'ensemble':
        return get_ensemble_top10(price_wt)[['rowID', 'survey_order', 'model_ranks']]
    else:
        print ("please enter 'rf', 'poly', or 'ensemble'")

In [ ]:
### Checking the property order is correct ###
# user_evaluation = pd.read_csv('../smartRE/survey/User_Evaluation_20180727.csv')
# df = pd.read_csv('../smartRE/sandbox/Simple Model with Integrated Dataframe/df_integrated_0714.csv')      
# property_list = ['20174233022047', '20174151021003', '20174216015034', \
#                      '20178706014038', '20174156016063', '20174339007052', \
#                      '20172841038054', '20172728020017', '20177564020019', '20174230008070']
# property_list = list(map(int, property_list))
# df_miniset = df[df['rowID'].isin(property_list)]  
# propertyIndex = dict(zip(property_list,range(len(property_list))))
# df_miniset['original_order'] = df_miniset['rowID'].map(propertyIndex) + 1
# df_miniset.sort_values(by = 'original_order',ascending = True)

In [12]:
def compare(model, userID, price_wt):
    df_merge = pd.merge(left = model_output_ranks(model, userID, price_wt), \
                        right = user_input(userID)[1], left_on='rowID', right_on='rowID')
    df_merge = df_merge.set_index('rowID')
    return df_merge.sort_values(by=['survey_order'], ascending=True)

In [13]:
def calculate_accuracy(model, userID, price_wt):
    df_compare = compare(model, userID, price_wt)
    correct_pair = 0
    correct_top = 0
    total_pair = 0
    rowID = sorted(list(df_compare.index))
    for i in rowID:
        for j in rowID:
            if i >= j:
                continue
            else:
                total_pair += 1
                if (df_compare.loc[i, 'model_ranks'] - df_compare.loc[j, 'model_ranks']) * \
                (df_compare.loc[i, 'user_ranks'] - df_compare.loc[j, 'user_ranks']) > 0:
                    correct_pair += 1
    pair_accuracy = correct_pair / total_pair
    
#     model_top3 = model_ranks[model_ranks.isin([1,2,3])]
#     for i in model_top3.index:
#         if (user_ranks.loc[i] <= 5).bool():
#             correct_top += 1
    user_top3 = df_compare[df_compare['user_ranks'] <= 3]
    for i in user_top3.index:
        if df_compare.loc[i, 'model_ranks'] <= 5:
            correct_top += 1
    top_accuracy = correct_top / 5
 
    return (pair_accuracy, top_accuracy)

In [14]:
# # price_wts = np.arange(0,2.0,0.5)
# price_wts = [0.01, 0.02]
# plt.figure(figsize=(10,40))
# count = 0
# for userID in tqdm(range(2,24)):
#     ax = plt.subplot(8, 3, count + 1)
#     plt.setp(ax, xticks=(price_wts), yticks=(np.arange(0,1.0,0.1)))
#     plt.plot(price_wts, calculate_accuracy('rf', userID, price_wts)[0])
#     plt.ylim([0,1.0])
#     count += 1
# plt.show()

In [15]:
def evaluate_models(model, price_wts):
    total_pair_accuracy, user_pair_accuracy, best_pair_accuracy, best_pair_wt = {}, {}, {}, {}
    total_top_accuracy, user_top_accuracy, best_top_accuracy, best_top_wt = {}, {}, {}, {}
    
    for price_wt in tqdm(price_wts):
        total_pair_accuracy[price_wt], total_top_accuracy[price_wt] = 0, 0
        
        for userID in range(2,len(user_evaluation)):
            pair_accuracy, top_accuracy = calculate_accuracy(model, userID, price_wt)
            total_pair_accuracy[price_wt] += pair_accuracy
            total_top_accuracy[price_wt] += top_accuracy
            user_pair_accuracy[(price_wt,userID)] = pair_accuracy
            user_top_accuracy[(price_wt,userID)] = top_accuracy
            
            if userID in best_pair_accuracy.keys():
                if pair_accuracy > best_pair_accuracy[userID]:
                    best_pair_accuracy[userID] = pair_accuracy
                    best_pair_wt[userID] = price_wt
            else:
                best_pair_accuracy[userID] = pair_accuracy
                best_pair_wt[userID] = price_wt
                
            if userID in best_top_accuracy.keys():
                if top_accuracy > best_top_accuracy[userID]:
                    best_top_accuracy[userID] = top_accuracy
                    best_top_wt[userID] = price_wt
            else:
                best_top_accuracy[userID] = top_accuracy
                best_top_wt[userID] = price_wt
        total_pair_accuracy[price_wt] = total_pair_accuracy[price_wt] / (len(user_evaluation) - 2)
        total_top_accuracy[price_wt] = total_top_accuracy[price_wt] / (len(user_evaluation) - 2)
        
    avg_pair_acc = max(list(total_pair_accuracy.values()))
    avg_top_acc = max(list(total_top_accuracy.values()))
    best_pair_wt_num = list(total_pair_accuracy.keys())[np.argmax(list(total_pair_accuracy.values()))]
    best_top_wt_num = list(total_top_accuracy.keys())[np.argmax(list(total_top_accuracy.values()))]

    
    LA, user_pair_acc, pair_best, pair_wt, user_top_acc, top_best, top_wt = [], [], [], [], [], [], []
    for userID in range(2,len(user_evaluation)):
        LA.append(user_input(userID)[2])
        user_pair_acc.append(user_pair_accuracy[(best_pair_wt_num, userID)])
        pair_best.append(best_pair_accuracy[userID])
        pair_wt.append(best_pair_wt[userID])
        
        user_top_acc.append(user_top_accuracy[(best_top_wt_num, userID)])
        top_best.append(best_top_accuracy[userID])
        top_wt.append(best_top_wt[userID])

    df_LA = pd.concat([pd.Series(LA), pd.Series(user_pair_acc), pd.Series(pair_best), pd.Series(pair_wt), \
                       pd.Series(user_top_acc), pd.Series(top_best), pd.Series(top_wt)], axis=1)
    df_LA.columns = ['LA', 'user_pair_accuracy', 'best_pair_accuracy', 'best_pair_wt', \
                     'user_top_accuracy', 'best_top_accuracy', 'best_top_wt']

    return [best_pair_wt_num, avg_pair_acc, best_top_wt_num, avg_top_acc, df_LA]

## EVALUATE MODELS

### Random Forest

In [41]:
price_wts = np.arange(0,10.0,0.05)

In [42]:
best_pair_wt, pair_acc, best_top_wt, top_acc, df_LA = evaluate_models('rf', price_wts)

100%|██████████| 200/200 [21:01<00:00,  6.31s/it]


In [43]:
print ('best pair wt =', best_pair_wt)
print ('avg pair accuracy =', pair_acc)
print ('best top wt =', best_top_wt)
print ('avg top accuracy =', top_acc)
print ('LA local pair accuracy =', df_LA[df_LA['LA']=='Yes']['user_pair_accuracy'].mean())
print ('Non-LA   pair accuracy =', df_LA[df_LA['LA']=='No']['user_pair_accuracy'].mean())
print ('LA local top accuracy =', df_LA[df_LA['LA']=='Yes']['user_top_accuracy'].mean())
print ('Non-LA   top accuracy =', df_LA[df_LA['LA']=='No']['user_top_accuracy'].mean())

best pair wt = 9.2
avg pair accuracy = 0.5888888888888888
best top wt = 6.7
avg top accuracy = 0.39166666666666683
LA local pair accuracy = 0.7234567901234568
Non-LA   pair accuracy = 0.508148148148148
LA local top accuracy = 0.4888888888888888
Non-LA   top accuracy = 0.3333333333333333


In [44]:
df_LA

,LA,user_pair_accuracy,best_pair_accuracy,best_pair_wt,user_top_accuracy,best_top_accuracy,best_top_wt
0,No,0.555556,0.711111,0.30,0.4,0.4,0.00
1,Yes,0.777778,0.777778,9.20,0.6,0.6,4.35
2,No,0.400000,0.577778,0.10,0.2,0.4,0.00
3,No,0.488889,0.511111,3.55,0.4,0.4,3.55
4,Yes,0.733333,0.733333,5.55,0.4,0.4,3.35
5,No,0.600000,0.733333,0.20,0.4,0.6,0.20
6,No,0.400000,0.600000,0.00,0.4,0.4,0.35
7,No,0.333333,0.711111,0.00,0.2,0.4,0.00
8,No,0.711111,0.711111,5.35,0.6,0.6,0.15
9,Yes,0.866667,0.866667,7.70,0.6,0.6,3.95


### Polynomial Regression

In [78]:
price_wts = [0.01, 0.1, 1, 10, 100, 1000, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11, 10**12, 10**13, 10**14, 10**15]

In [58]:
price_wts = np.arange(-100.0,100.0,0.5)

In [79]:
best_pair_wt, pair_acc, best_top_wt, top_acc, df_LA = evaluate_models('poly', price_wts)

100%|██████████| 18/18 [01:50<00:00,  6.16s/it]


In [80]:
print ('best pair wt =', best_pair_wt)
print ('avg pair accuracy =', pair_acc)
print ('best top wt =', best_top_wt)
print ('avg top accuracy =', top_acc)
print ('LA local pair accuracy =', df_LA[df_LA['LA']=='Yes']['user_pair_accuracy'].mean())
print ('Non-LA   pair accuracy =', df_LA[df_LA['LA']=='No']['user_pair_accuracy'].mean())
print ('LA local top accuracy =', df_LA[df_LA['LA']=='Yes']['user_top_accuracy'].mean())
print ('Non-LA   top accuracy =', df_LA[df_LA['LA']=='No']['user_top_accuracy'].mean())

best pair wt = 1000000000000
avg pair accuracy = 0.5842592592592591
best top wt = 1000000000000
avg top accuracy = 0.38333333333333347
LA local pair accuracy = 0.7283950617283951
Non-LA   pair accuracy = 0.4977777777777777
LA local top accuracy = 0.4888888888888888
Non-LA   top accuracy = 0.32000000000000006


In [81]:
df_LA

,LA,user_pair_accuracy,best_pair_accuracy,best_pair_wt,user_top_accuracy,best_top_accuracy,best_top_wt
0,No,0.511111,0.511111,1000000.00,0.4,0.4,1000000.00
1,Yes,0.777778,0.800000,1000000.00,0.6,0.6,100000.00
2,No,0.400000,0.600000,0.01,0.2,0.2,0.01
3,No,0.488889,0.488889,1000000.00,0.4,0.4,0.01
4,Yes,0.733333,0.755556,10000.00,0.4,0.6,0.01
5,No,0.600000,0.600000,1000000.00,0.4,0.4,100000.00
6,No,0.400000,0.422222,100000.00,0.4,0.4,100000.00
7,No,0.333333,0.533333,0.01,0.2,0.4,0.01
8,No,0.711111,0.711111,1000000.00,0.6,0.6,1000000.00
9,Yes,0.866667,0.866667,10000000.00,0.6,0.6,100000.00


### Ensemble

In [83]:
price_wts = np.arange(0,10.0,0.05)

In [84]:
best_pair_wt, pair_acc, best_top_wt, top_acc, df_LA = evaluate_models('ensemble', price_wts)

100%|██████████| 200/200 [42:19<00:00, 12.70s/it]


In [85]:
print ('best pair wt =', best_pair_wt)
print ('avg pair accuracy =', pair_acc)
print ('best top wt =', best_top_wt)
print ('avg top accuracy =', top_acc)
print ('LA local pair accuracy =', df_LA[df_LA['LA']=='Yes']['user_pair_accuracy'].mean())
print ('Non-LA   pair accuracy =', df_LA[df_LA['LA']=='No']['user_pair_accuracy'].mean())
print ('LA local top accuracy =', df_LA[df_LA['LA']=='Yes']['user_top_accuracy'].mean())
print ('Non-LA   top accuracy =', df_LA[df_LA['LA']=='No']['user_top_accuracy'].mean())

best pair wt = 9.9
avg pair accuracy = 0.5166666666666667
best top wt = 4.0
avg top accuracy = 0.35000000000000003
LA local pair accuracy = 0.6419753086419753
Non-LA   pair accuracy = 0.44148148148148153
LA local top accuracy = 0.4666666666666667
Non-LA   top accuracy = 0.2800000000000001


In [86]:
df_LA

,LA,user_pair_accuracy,best_pair_accuracy,best_pair_wt,user_top_accuracy,best_top_accuracy,best_top_wt
0,No,0.422222,0.533333,0.20,0.2,0.4,5.70
1,Yes,0.600000,0.644444,3.35,0.4,0.6,4.35
2,No,0.400000,0.555556,0.20,0.2,0.4,0.00
3,No,0.422222,0.422222,3.55,0.4,0.4,2.95
4,Yes,0.688889,0.711111,3.05,0.4,0.4,0.00
5,No,0.488889,0.533333,3.30,0.4,0.4,0.15
6,No,0.377778,0.488889,0.00,0.4,0.4,0.35
7,No,0.355556,0.577778,0.00,0.2,0.4,0.00
8,No,0.688889,0.688889,5.75,0.4,0.6,0.00
9,Yes,0.666667,0.666667,5.95,0.4,0.6,5.95


### Visualize Ranks

In [87]:
pd.concat([compare('rf', 2, 9.2), compare('rf', 10, 9.2)], axis = 1)

,survey_order,model_ranks,user_ranks,survey_order,model_ranks,user_ranks
rowID,,,,,,
20174233022047,1,7,7,1,6,8
20174151021003,2,3,9,2,3,9
20174216015034,3,4,3,3,4,4
20178706014038,4,6,2,4,8,10
20174156016063,5,5,1,5,5,2
20174339007052,6,1,5,6,1,1
20172841038054,7,10,4,7,10,7
20172728020017,8,2,6,8,2,3
20177564020019,9,9,8,9,9,6
